In [18]:
# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 
from linebot.exceptions import (
    InvalidSignatureError
)

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageMessage, AudioMessage
)

# 載入設定檔

import json
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")


# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/" )

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [19]:
from aip import AipSpeech
import json
import time

""" 你的 APPID AK SK """
APP_ID = '17610496'
API_KEY = '3HxD7AjTwLrjEdWzjno97lbl'
SECRET_KEY = 'rPBbzb3enl2EA0cK0n98zHktbWDqYPt8'

begin = time.time()
client = AipSpeech(APP_ID, API_KEY, SECRET_KEY)


# 读取文件
def get_file_content(filepath):
    with open(filepath, 'rb') as fp:
        return fp.read()

In [20]:
@handler.add(MessageEvent, message=AudioMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='沒事亂傳訊息，害我又要加班'))
    message_content = line_bot_api.get_message_content(event.message.id)
    with open('./audios/'+event.message.id+".wav", 'wb') as fd:           ####格式問題:m4a,wav,mp4都試過了
        for chunk in message_content.iter_content():
            fd.write(chunk)
            
    # 识别本地文件
    voicefile = event.message.id+".wav"
    base = "./audios/"
    # print(base+voicefile)
    result = client.asr(get_file_content(base+voicefile), 'wav', 16000, {
        'dev_pid': 1536,
    })
    # print(result)
    print("Request time cost %f" % (time.time() - begin))
    if result['err_no'] == 0:
        ofile = "result_%s.txt" % (voicefile)
        with open(ofile, "w", encoding="utf-8") as of:
            json.dump(result, of, ensure_ascii=False)
    else:
        print(result['err_msg'], result['err_no'])

In [21]:
# from aip import AipSpeech
# import json
# import time

# """ 你的 APPID AK SK """
# APP_ID = '17605237'
# API_KEY = '5yp7MfZ6plHeSXTxVQlpZznF'
# SECRET_KEY = 'DxnTt3LnpumGnpQdYjZWWu34NmDnyOQG'

# begin = time.time()
# client = AipSpeech(APP_ID, API_KEY, SECRET_KEY)


# # 读取文件
# def get_file_content(filepath):
#     with open(filepath, 'rb') as fp:
#         return fp.read()


# # 识别本地文件
# voicefile = event.message.id+".wav"
# base = "./audios/"
# # print(base+voicefile)
# result = client.asr(get_file_content(base+voicefile), 'wav', 16000, {
#     'dev_pid': 1536,
# })
# # print(result)
# print("Request time cost %f" % (time.time() - begin))
# if result['err_no'] == 0:
#     ofile = "result_%s.txt" % (voicefile)
#     with open(ofile, "w", encoding="utf-8") as of:
#         json.dump(result, of, ensure_ascii=False)
# else:
#     print(result['err_msg'], result['err_no'])

# print("完成了")

In [ ]:
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Oct/2019 21:59:16] "POST / HTTP/1.1" 200 -


Request time cost 43.250000
speech quality error. 3301
